### Overview of the dataset:
The attached dataset is a baseline – aggregated data holding key performance metrics across different selling points for different products. Each product can be represented in different selling points (DistributionUnit) and have different prices depending on where is being sold.

### Explanation of some fields:


BaselineDetailID – Id of the product in the DistributionUnit

BaselineID – ID of current baseline.

ProductID – General ID of the Product

DistributionUnit – sales channel.

NSRperUC -  Net Sales Revenue Per Unit Case, presented only for TCCC Manufacturer.

COGS – Cost of goods sold

ExitRate – Percentage of the volume that will be lost if the product is not represented in the store

### Overview of the task:
Create an executive summary for a provided baseline that can be shown to a market owner to help him better understand the market structure and TCCC's position. Below is the list of the visualizations that might be helpful, but this list is not a hard restriction.

### High-level aggregations:

- top Manufacturers

- top Brands by Volume and Manufacturer

- top Categories by Volume

- top Brand PackSize by Channel and Manufacturer by Volume for all, by Revenue for TCCC

### Competitor selection:

- for the top 3 TCCC brands: the top 3 SKUs from each brand present the most suitable competitor SKU.
Describe what criteria are used to define competitor SKU and why. How do TCCC SKUs perform in contrast to competitor SKUs?

### Product Performance:

- what are the key products that must always be presented in the store to avoid loss of sales? (Hist Exit Rates)
- what are the key products for which price change can lead to an unprofitable loss of sales? (Hint: Elasticity)

The result should be presented in a jupyter-notebook with clear, interactive visualizations. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('products-metrics/DemoBaseline.xlsx')
df.head()

,BaselineDetailID,BaselineID,ProductID,DistributionUnit,CurrencyCode,Price,NSRperUC,EnvironmentalTaxPerUC,OperatingCOGSperUC,WarehouseCostPerUC,...,ExitRate,RatioNonOperatingCOGStoNSR,Distribution,Description,Manufacturer,Category,Brand,Flavor,PackSize,Packaging
0,816018,451,358130194,SOUTH,484,62.8757,0.00000,0,0.000,0,...,12.71589,0,0.00100,ADAS_PEPSICO_SEVEN UP LIMONADA_LIMON_1500ML_EN...,PEPSICO,ADAS,SEVEN UP LIMONADA,LIMON,1500,BOTELLA DE PLASTICO_NR
1,816019,451,358130194,EAST,484,65.9368,0.00000,0,0.000,0,...,12.71589,0,0.00100,ADAS_PEPSICO_SEVEN UP LIMONADA_LIMON_1500ML_EN...,PEPSICO,ADAS,SEVEN UP LIMONADA,LIMON,1500,BOTELLA DE PLASTICO_NR
2,816020,451,358130194,WEST,484,62.8757,0.00000,0,0.000,0,...,12.71589,0,9.00000,ADAS_PEPSICO_SEVEN UP LIMONADA_LIMON_1500ML_EN...,PEPSICO,ADAS,SEVEN UP LIMONADA,LIMON,1500,BOTELLA DE PLASTICO_NR
3,816021,451,358130194,NORTH,484,62.8757,0.00000,0,0.000,0,...,12.71589,0,0.00100,ADAS_PEPSICO_SEVEN UP LIMONADA_LIMON_1500ML_EN...,PEPSICO,ADAS,SEVEN UP LIMONADA,LIMON,1500,BOTELLA DE PLASTICO_NR
4,816014,451,358130195,SOUTH,484,126.7034,75.13333,0,22.429,0,...,9.63984,0,0.12124,ADAS_COCA-COLA COMPANY_DEL VALLE Y NADA_NARANJ...,TCCC,ADAS,DEL VALLE Y NADA,NARANJA,600,BOTELLA DE PLASTICO_NR


In [3]:
df.shape

(2843, 24)

In [4]:
df.columns

Index(['BaselineDetailID', 'BaselineID', 'ProductID', 'DistributionUnit',
       'CurrencyCode', 'Price', 'NSRperUC', 'EnvironmentalTaxPerUC',
       'OperatingCOGSperUC', 'WarehouseCostPerUC', 'DistributionCostPerUC',
       'TransactionsPerUC', 'Volume', 'PriceElasticity', 'ExitRate',
       'RatioNonOperatingCOGStoNSR', 'Distribution', 'Description',
       'Manufacturer', 'Category', 'Brand', 'Flavor', 'PackSize', 'Packaging'],
      dtype='object')

# High-level aggregations:

### - top Manufacturers


In [5]:
df['Manufacturer'].value_counts().head()

TCCC              930
GRUPO PENAFIEL    348
PEPSICO           310
AGA DE MEXICO     226
DANONE            167
Name: Manufacturer, dtype: int64

### - top Brands by Volume and Manufacturer


In [6]:
brand_manuf = df.groupby(['Manufacturer', 'Brand']) \
                .agg({'Volume': 'sum'}) \
                .sort_values(by='Volume', ascending=False)
brand_manuf.head()

,,Volume
Manufacturer,Brand,
TCCC,COCA COLA,1.041345e+09
PEPSICO,PEPSI COLA,4.326280e+07
TCCC,CIEL,2.718835e+07
GRUPO GEPP,E-PURA,2.181769e+07
DANONE,BONAFONT,1.218342e+07



### - top Categories by Volume


In [7]:
categ = df.groupby('Category') \
          .agg({'Volume': 'sum'}) \
          .sort_values(by='Volume', ascending=False)
categ.head()

,Volume
Category,
COLAS,1.107281e+09
AGUA EMBOTELLADA,6.580332e+07
R. FRUTALES,6.289466e+07
NARANJADAS,1.349067e+07
AGUA MINERAL NATURAL,7.764272e+06


### - top Brand PackSize by Channel and Manufacturer by Volume for all, by Revenue for TCCC

In [8]:
# add revenue column
df['Revenue'] = df['Price'] * df['Volume']
# define DataFrame of TCCC
tccc_df = df[df['Manufacturer'] == "TCCC"]

In [9]:
# group the TCCC dataframe by PackSize, Brand, Channel and calculate the sum of Revenue
tccc_packsizes = df.groupby(['PackSize', 'Brand', 'DistributionUnit']) \
                   .agg({'Revenue': 'sum'}) \
                   .reset_index()
tccc_packsizes.head(3)

,PackSize,Brand,DistributionUnit,Revenue
0,1,AGGREGATED,EAST,9.298097e+07
1,1,AGGREGATED,NORTH,9.987632e+07
2,1,AGGREGATED,SOUTH,5.332763e+07


In [10]:
# group the dataframe by PackSize, Brand, Channel and Manufacturer and calculate the sum of Volume
packsizes = df.groupby(['PackSize', 'Brand', 'DistributionUnit', 'Manufacturer']) \
              .agg({'Volume': 'sum'}) \
              .reset_index()
packsizes.head(3)

,PackSize,Brand,DistributionUnit,Manufacturer,Volume
0,1,AGGREGATED,EAST,AGGREGATED,8.977673e+05
1,1,AGGREGATED,NORTH,AGGREGATED,1.020671e+06
2,1,AGGREGATED,SOUTH,AGGREGATED,5.536072e+05


In [11]:
# get top TCCC PackSizes by Revenue
tccc_top_ps = tccc_packsizes.groupby(['Brand', 'DistributionUnit']) \
                            .agg({'Revenue': 'max'}) \
                            .reset_index()
tccc_top_ps.head(3)

,Brand,DistributionUnit,Revenue
0,ACAPULCO,NORTH,204018.422967
1,ACAPULCO,SOUTH,88100.250716
2,ACAPULCO,WEST,110162.618869


In [12]:
# get top PackSizes by Volume
top_ps = packsizes.groupby(['Brand', 'DistributionUnit', 'Manufacturer']) \
                  .agg({'Volume': 'max'}) \
                  .reset_index()
top_ps.head(3)

,Brand,DistributionUnit,Manufacturer,Volume
0,ACAPULCO,NORTH,LA CONCORDIA,5620.38868
1,ACAPULCO,SOUTH,LA CONCORDIA,2427.02421
2,ACAPULCO,WEST,LA CONCORDIA,3034.80797


In [13]:
# join TCCC tables to get PackSize
tccc_pack_size = tccc_top_ps.merge(tccc_packsizes, on='Revenue', suffixes=('_', '')).loc[:, 'Revenue':'DistributionUnit']
tccc_pack_size.head()

,Revenue,PackSize,Brand,DistributionUnit
0,204018.422967,1000,ACAPULCO,NORTH
1,88100.250716,1000,ACAPULCO,SOUTH
2,110162.618869,1000,ACAPULCO,WEST
3,32831.974312,330,ACAPULCOCO,NORTH
4,7269.902674,330,ACAPULCOCO,SOUTH


In [14]:
# join tables to get PackSize
pack_size = top_ps.merge(packsizes, on='Volume', suffixes=('_', '')).loc[:, 'Volume':'Manufacturer']
pack_size.head()

,Volume,PackSize,Brand,DistributionUnit,Manufacturer
0,5620.38868,1000,ACAPULCO,NORTH,LA CONCORDIA
1,2427.02421,1000,ACAPULCO,SOUTH,LA CONCORDIA
2,3034.80797,1000,ACAPULCO,WEST,LA CONCORDIA
3,113.39968,330,ACAPULCOCO,NORTH,CALAHUA
4,25.10981,330,ACAPULCOCO,SOUTH,CALAHUA


### - Top Manufacturer by Volume

In [15]:
manuf = df.groupby(['Manufacturer']) \
          .agg({'Volume': 'sum'}) \
          .sort_values(by='Volume', ascending=False)
manuf.head()

,Volume
Manufacturer,
TCCC,1.119014e+09
PEPSICO,6.092470e+07
GRUPO GEPP,2.503962e+07
AGA DE MEXICO,2.074967e+07
DANONE,1.218342e+07


# Competitor selection:

- for the top 3 TCCC brands: the top 3 SKUs from each brand present the most suitable competitor SKU.
Describe what criteria are used to define competitor SKU and why. How do TCCC SKUs perform in contrast to competitor SKUs?

## Top 3 TCCC brands 
Brands are chosen by Volume

In [16]:
tccc_top = df[df['Manufacturer'] == 'TCCC'].groupby('Brand') \
            .agg({'Volume': 'sum'}) \
            .sort_values(by='Volume', ascending=False) \
            .reset_index() \
            .head(3)

In [17]:
tccc_top

,Brand,Volume
0,COCA COLA,1.041345e+09
1,CIEL,2.718835e+07
2,SPRITE,6.284246e+06


## Top products for each top brand of TCCC
Products are also taken by Volume

In [18]:
tccc_top_products = {}  # dict with top TCCC brands : dataframe with top products by Volume
for brand in tccc_top['Brand']:
    tccc_top_products[brand] = df[df['Brand'] == brand] \
                                .groupby(['ProductID', 'Category']) \
                                .agg({'Volume': 'sum'}) \
                                .sort_values(by='Volume', ascending=False) \
                                .reset_index() \
                                .head(3)
tccc_top_products

{'COCA COLA':    ProductID Category        Volume
 0  358147329    COLAS  6.337097e+08
 1  358130403    COLAS  4.236299e+07
 2  358130405    COLAS  2.324601e+07,
 'CIEL':    ProductID          Category        Volume
 0  358130211  AGUA EMBOTELLADA  1.472050e+07
 1  358130213  AGUA EMBOTELLADA  4.846272e+06
 2  358130215  AGUA EMBOTELLADA  3.984154e+06,
 'SPRITE':    ProductID     Category        Volume
 0  358130586  R. FRUTALES  1.597091e+06
 1  358130587  R. FRUTALES  1.258110e+06
 2  358130590  R. FRUTALES  9.753609e+05}

### DataFrame of TCCC competitors

In [19]:
no_tccc_df = df[(df['Manufacturer'] != 'TCCC')]
no_tccc_df.head(3)

,BaselineDetailID,BaselineID,ProductID,DistributionUnit,CurrencyCode,Price,NSRperUC,EnvironmentalTaxPerUC,OperatingCOGSperUC,WarehouseCostPerUC,...,RatioNonOperatingCOGStoNSR,Distribution,Description,Manufacturer,Category,Brand,Flavor,PackSize,Packaging,Revenue
0,816018,451,358130194,SOUTH,484,62.8757,0.0,0,0.0,0,...,0,0.001,ADAS_PEPSICO_SEVEN UP LIMONADA_LIMON_1500ML_EN...,PEPSICO,ADAS,SEVEN UP LIMONADA,LIMON,1500,BOTELLA DE PLASTICO_NR,4.334210e+06
1,816019,451,358130194,EAST,484,65.9368,0.0,0,0.0,0,...,0,0.001,ADAS_PEPSICO_SEVEN UP LIMONADA_LIMON_1500ML_EN...,PEPSICO,ADAS,SEVEN UP LIMONADA,LIMON,1500,BOTELLA DE PLASTICO_NR,7.027897e+05
2,816020,451,358130194,WEST,484,62.8757,0.0,0,0.0,0,...,0,9.000,ADAS_PEPSICO_SEVEN UP LIMONADA_LIMON_1500ML_EN...,PEPSICO,ADAS,SEVEN UP LIMONADA,LIMON,1500,BOTELLA DE PLASTICO_NR,1.000620e+06


## Top competitors for COCA COLA SKUs
Since all COCA COLA (CC) products are in the same category, the competitors' products are the same for each CC product

In [20]:
tccc_top_products['COCA COLA']

,ProductID,Category,Volume
0,358147329,COLAS,6.337097e+08
1,358130403,COLAS,4.236299e+07
2,358130405,COLAS,2.324601e+07


In [21]:
# get products that are in the same category that CC SKUs, and get top-3 of them by Volume
cc_competitors = no_tccc_df[(no_tccc_df['Brand'] != 'COCA COLA') & (no_tccc_df['Category'] == 'COLAS')] \
                            .groupby('ProductID') \
                            .agg({'Volume': 'sum'}) \
                            .sort_values(by='Volume', ascending=False) \
                            .reset_index() \
                            .head(3)
cc_competitors

,ProductID,Volume
0,358130417,1.458619e+07
1,358130420,1.173057e+07
2,358130416,7.835953e+06


In [22]:
# list of COCA COLA competitor brands
cc_comp_mask = (no_tccc_df['ProductID'].isin(cc_competitors['ProductID']))
no_tccc_df[cc_comp_mask]['Brand'].unique()

array(['PEPSI COLA'], dtype=object)

As we see, the only competitor to COCA COLA products is PEPSI COLA brand. 
<br>The volume of PEPSI COLA products is much (~ 1.5-40 times) less than the volume of COCA COLA products.

## Top competitors for CIEL SKUs
Since all CIEL products are in the same category, the competitors' products are the same for each CIEL product

In [23]:
tccc_top_products['CIEL']

,ProductID,Category,Volume
0,358130211,AGUA EMBOTELLADA,1.472050e+07
1,358130213,AGUA EMBOTELLADA,4.846272e+06
2,358130215,AGUA EMBOTELLADA,3.984154e+06


In [24]:
# get products that are in the same category that CIEL SKUs, and get top-3 of them by Volume
ciel_competitors = no_tccc_df[(no_tccc_df['Brand'] != 'CIEL') 
                               & (no_tccc_df['Category'] == 'AGUA EMBOTELLADA')].groupby('ProductID') \
                                .agg({'Volume': 'sum'}) \
                                .sort_values(by='Volume', ascending=False) \
                                .reset_index() \
                                .head(3)
ciel_competitors

,ProductID,Volume
0,358130212,1.406443e+07
1,358130214,5.361399e+06
2,358130220,2.514826e+06


In [25]:
# list of CIEL competitor brands
ciel_comp_mask = (no_tccc_df['ProductID'].isin(ciel_competitors['ProductID']))
no_tccc_df[ciel_comp_mask]['Brand'].unique()

array(['E-PURA', 'BONAFONT'], dtype=object)

CIEL competitors are E-PURA and BONAFONT.
<br>CIEL competitors are almost on the same level based on Volume metric.

## Top competitors for SPRITE SKUs
Since all SPRITE products are in the same category, the competitors' products are the same for each SPRITE product

In [26]:
tccc_top_products['SPRITE']

,ProductID,Category,Volume
0,358130586,R. FRUTALES,1.597091e+06
1,358130587,R. FRUTALES,1.258110e+06
2,358130590,R. FRUTALES,9.753609e+05


In [27]:
# get products that are in the same category that SPRITE SKUs, and get top-3 of them by Volume
sprite_competitors = no_tccc_df[(no_tccc_df['Brand'] != 'SPRITE') 
                               & (no_tccc_df['Category'] == 'R. FRUTALES')].groupby('ProductID') \
                                .agg({'Volume': 'sum'}) \
                                .sort_values(by='Volume', ascending=False) \
                                .reset_index() \
                                .head(3)
sprite_competitors

,ProductID,Volume
0,358130584,7.704361e+06
1,358130593,2.742335e+06
2,358130707,2.415774e+06


In [28]:
# list of SPRITE competitor brands
sprite_comp_mask = (no_tccc_df['ProductID'].isin(sprite_competitors['ProductID']))
no_tccc_df[sprite_comp_mask]['Brand'].unique()

array(['SQUIRT', 'SEVEN UP', 'AGA'], dtype=object)

In [29]:
no_tccc_df[no_tccc_df['ProductID'] == 358130584]['Brand'].unique()

array(['SQUIRT'], dtype=object)

SPRITE competitors are SQUIRT, SEVEN UP, and AGA.
<br>SPRITE competitors are better than SPRITE based on Volume metric. The top Volume of SQUIRT is 4.8 times greater than the top Volume of SPRITE

# Product Performance:

- what are the key products that must always be presented in the store to avoid loss of sales? (Hist Exit Rates)


In [30]:
# add the column of the absolute amount of the lost volume
df['LostVolume'] = df['Volume'] * df['ExitRate']

In [31]:
# get top-5 products with brands with the biggest volume loss
df.groupby(['ProductID', 'Brand']) \
  .agg({'LostVolume': 'sum'}) \
  .sort_values('LostVolume', ascending=False) \
  .head()

,,LostVolume
ProductID,Brand,
358147329,COCA COLA,9.093735e+09
358130403,COCA COLA,5.982561e+08
358130405,COCA COLA,2.854950e+08
358146312,COCA COLA,2.640400e+08
358146307,COCA COLA,2.640400e+08



- what are the key products for which price change can lead to an unprofitable loss of sales? (Hint: Elasticity)

In [32]:
# add the column of the loss of revenue
df['RevenueLoss'] = df['Revenue'] * df['PriceElasticity']

In [33]:
df.groupby(['ProductID', 'Brand']) \
  .agg({'RevenueLoss':'sum'}) \
  .sort_values('RevenueLoss') \
  .head()

,,RevenueLoss
ProductID,Brand,
358147329,COCA COLA,-8.053183e+10
358147326,COCA COLA,-3.588000e+09
358130403,COCA COLA,-2.487830e+09
358146312,COCA COLA,-2.351520e+09
358146307,COCA COLA,-2.351520e+09
